In [ ]:
using Unitful
include("../src/AutoSolver.jl")

In [ ]:
# include("../../AstroKit.jl/src/AstroKit.jl")
include("../../SciBase.jl/src/SciBase.jl")

In [ ]:
# # Displacement
# disp(x0,x) = x-x0
# dispConstAcc1(v0,a,t) = v0*t+1/2*a*t^2
# dispConstAcc2(v0,v,t) = ((v+v0)/2)*t
# # Velocity
# avgVel(x0,x,t0,t) = (x-x0)/(t-t0)
# velConstAcc1(v0,a,t) = v0+a*t
# velConstAcc2(x0,x,v0,a) = sqrt(v0^2+2*a*(x-x0))
# # Acceleration
# accConst(x0,x,v0,v) = (v^2-v0^2)/(2*(x-x0))

In [ ]:
# # ve = 10u"km/s"
# # mi = 200u"kg"
# # mf = 80u"kg"
# # f = 2.4u"GHz"
# x0 = 1.0u"m"
# x = 2.0u"m"
# v0 = 1.0u"m/s"
# v = 2.0u"m/s"

# mods = Module[]
# AutoSolver.submodules(physics, mods)
# # mods = [Main]
# # mods = [AstroKit]

# outputs = AutoSolver.autoSolveOutputs(mods)


In [ ]:
# mods = Module[]
# funcs = Dict()
# AutoSolver.submodules(physics, mods)
# AutoSolver.moduleDict(Main.physics.kinematics, funcs)
# mods

In [ ]:
# out_d, in_d = AutoSolver.varExplodeDict(funcs)
# inp_map, out_map = AutoSolver.crossVarMap(in_d,out_d)
# inp_map

In [ ]:
# import Pkg; Pkg.add("XLSX")

In [ ]:
using XLSX

# Create Spreadsheet Template

In [ ]:
function autoSolveTemplate(mod::Module;filePath::String="",overwrite=false)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    mod_name = AutoSolver.modName(mod)
    
    if isfile("$(filePath)AS_$(mod_name).xlsx") && overwrite == false
        println("ERROR: $(filePath)AS_$(mod_name).xlsx already exists")
        return
    elseif !isdir(filePath) && filePath != ""
        mkdir(filePath)
    end

    XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="wr") do xf
        XLSX.addsheet!(xf)
        XLSX.addsheet!(xf)
        sh1 = xf[1]
        sh2 = xf[2]
        sh3 = xf[3]
        XLSX.rename!(sh1, "Input Map")
        XLSX.rename!(sh2, "Inputs")
        XLSX.rename!(sh3, "Outputs")

        sh1[1,1] = "Module"
        sh1[1,2] = "Function"
        sh1[1,3] = "Input Variable"
        sh1[1,4] = "Input Map"

        sh2[1,1] = "Input Variable"
        sh2[1,2] = "Input"
        sh2[1,3] = "Units"

        sh3[1,1] = "Module"
        sh3[1,2] = "Function"
        sh3[1,3] = "Output"
        sh3[1,4] = "Units"

        r1 = 2
        r2 = 2
        r3 = 2
        new_input = 0
        
        for submod in mods
            funcs = Dict()
            AutoSolver.moduleDict(submod, funcs)
            submod_name = AutoSolver.modName(submod)

            for func in funcs 
                sh1[r1,1] = submod_name
                sh1[r1,2] = string(func[1])

                for i = 1:length(func[2])
                    sh1[r1,3] = string(func[2][i])
                    sh1[r1,4] = string(func[2][i])
                    r1+=1

                    for j = 2:r2
                        if string(sh2[j,1]) == string(func[2][i])
                            new_input = 0
                            break
                        else
                            new_input = 1
                        end
                    end
                    if new_input == 1
                        sh2[r2,1] = string(func[2][i])
                        r2+=1
                    end
                end
            end

            for func in funcs      
                sh3[r3,1] = submod_name
                sh3[r3,2] = string(func[1])
                sh3[r3,3] = "-"
                sh3[r3,4] = "-"
                r3+=1
            end
        end
    end
end

In [ ]:
autoSolveTemplate(physics, filePath="C:/Users/ses/Documents/AutoSolver/", overwrite=false)

# Update Spreadsheet Mapping

In [ ]:
function inputMapFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Input Map"]
    inp_map = Dict()
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if (i == 1 || ismissing(r[4])); continue; end
        if !haskey(inp_map, Symbol(r[3])); inp_map[Symbol(r[3])] = []; end
        if !(Symbol(r[4]) in inp_map[Symbol(r[3])]); push!(inp_map[Symbol(r[3])],Symbol(r[4])); end
    end

    return inp_map
end

In [ ]:
function inputsFromXLSX(xf::XLSX.XLSXFile)
    sh = xf["Inputs"]
    
    for (i,r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        if ismissing(@eval $(r[2])); continue; end
        if ismissing(@eval $(r[3]))
            @eval $(Symbol(r[1])) = (@eval $(r[2]))
        else
            @eval $(Symbol(r[1])) = (@eval $(r[2])) * (@eval @u_str $(r[3]))
        end
    end
end

In [ ]:
function outputToXLSX(xf::XLSX.XLSXFile, outputs::Dict)
    sh = xf["Outputs"]
  
    for (i, r) in enumerate(XLSX.eachrow(sh))
        if i == 1; continue; end
        func = Symbol(r[2])
        if haskey(outputs,func)
            sh[i,3] = string(ustrip(outputs[func]))
            sh[i,4] = string(unit(outputs[func]))
        else
            sh[i,3] = "-"
            sh[i,4] = "-"
        end
    end
end

In [ ]:
function autoSolveXLSX(mod::Module, filePath::String)
    inp_map = Dict()
    mod_name = AutoSolver.modName(mod)

    inp_map = XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
        inp_map = inputMapFromXLSX(xf)
        inputsFromXLSX(xf)
        return inp_map
    end

    mods = Module[]
    AutoSolver.submodules(mod, mods)

    outputs = AutoSolver.autoSolveOutputs(mods, inp_map = inp_map)

    XLSX.openxlsx("$(filePath)AS_$(mod_name).xlsx", mode="rw") do xf
        outputToXLSX(xf,outputs)
    end
end

In [ ]:
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoSolveXLSX(mod, filePath)

# AutoTester

In [ ]:
function autoTestTemplate(mod::Module;filePath::String="",overwrite=false)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    mod_name = AutoSolver.modName(mod)
    num_tests = 5
    
    if isfile("$(filePath)AT_$(mod_name).xlsx") && overwrite == false
        println("ERROR: $(filePath)AT_$(mod_name).xlsx already exists")
        return
    elseif !isdir(filePath) && filePath != ""
        mkdir(filePath)
    end
    
    if !isfile("$(filePath)AT_$(mod_name).xlsx")
        XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="w") do xf
        end
    end

    XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="rw") do xf
        sh1 = xf[1]
        XLSX.rename!(sh1, "Test Inputs")

        sh1[1,1] = "Module"
        sh1[1,2] = "Function"
        sh1[1,3] = "Input Vars"
        
        for i = 1:num_tests
            sh1[1,3*i+1] = "Inputs $(i)"
            sh1[1,3*i+2] = "Exp Output $(i)"
            sh1[1,3*i+3] = "Tol $(i) [%]"
        end

        r1 = 2
        new_input = 0
        
        for submod in mods
            funcs = Dict()
            AutoSolver.moduleDict(submod, funcs)
            submod_name = AutoSolver.modName(submod)

            for func in funcs
                args = string(func[2])
                sh1[r1,1] = submod_name
                sh1[r1,2] = string(func[1])
                sh1[r1,3] = string(strip(string(args[5:end]),[']']))
                r1+=1
            end
        end
    end
end

In [ ]:
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoTestTemplate(mod, filePath=filePath, overwrite=false)

In [ ]:
function autoTestXLSX(mod::Module;filePath::String="")
    mod_name = AutoSolver.modName(mod)
    mods = Module[]
    AutoSolver.submodules(mod, mods) 
    
    XLSX.openxlsx("$(filePath)AT_$(mod_name).xlsx", mode="rw") do xf
        sh = xf["Test Inputs"]
        
        for (i,r) in enumerate(XLSX.eachrow(sh))
            if i == 1; continue; end
            if ismissing(@eval $(r[4])); continue; end
#             if ismissing(@eval $(r[3]))
#                 @eval $(Symbol(r[1])) = (@eval $(r[2]))
#             else
#                 @eval $(Symbol(r[1])) = (@eval $(r[2])) * (@eval @u_str $(r[3]))
#             end
            println(r[1],", ",r[2],", [",r[4],"], ",r[5],", ",r[6])
            args = split(r[4],',')
            arg_arr = []
            for arg in args
                arg_in = split(strip(arg)," ")
                val = parse(Int,string(arg_in[1]))
                units = @eval @u_str $(arg_in[2])
                var = val * units
                push!(arg_arr,var)
            end
            
#             println(Module(Symbol(r[1])),".",Symbol(r[2]))
#             println("Inputs: ",arg_arr)
#             println("Expected Output: ",r[5])
            
            for mod_s in mods
                if string(mod_s) == string((@eval $(Module(Symbol(r[1])))))
                    output = AutoSolver.evalVariableInputs(mod_s,Symbol(r[2]),arg_arr)
                    println("Actual Output: ", output)
                    if string(output) == string(r[5])
                        println("Test Pass\n")
                    else
                        println("Test Failed\n")
                    end
                end
            end
#             result = (@eval $(Module(Symbol(r[1]))).$(Symbol(r[2]))($(arg_arr)...))
        end
    end
end

In [ ]:
using Test
filePath="C:/Users/ses/Documents/AutoSolver/"
mod = physics

autoTestXLSX(mod, filePath=filePath)

In [ ]:
Main.physics.dynamics.F(1,2)
# Unitful.promote_to_derived()
# promote(1u"J",1.0u"kg*m^2/s^2")[2]

# AutoGenerate Database

In [ ]:
# using Pkg
# Pkg.add("PDFIO")

# """
# ​```
#     getPDFText(src, out) -> Dict 
# ​```
# - src - Input PDF file from where text is to be extracted
# - out - Output TXT file where the output will be written
# return - A dictionary containing metadata of the document
# """
# function getPDFText(src, out)
#     # handle that can be used for subsequence operations on the document.
#     doc = pdDocOpen(src)
    
#     # Metadata extracted from the PDF document. 
#     # This value is retained and returned as the return from the function. 
#     docinfo = pdDocGetInfo(doc) 
#     open(out, "w") do io
    
#         # Returns number of pages in the document       
#         npage = pdDocGetPageCount(doc)

#         for i=1:npage
        
#             # handle to the specific page given the number index. 
#             page = pdDocGetPage(doc, i)
            
#             # Extract text from the page and write it to the output file.
#             pdPageExtractText(io, page)

#         end
#     end
#     # Close the document handle. 
#     # The doc handle should not be used after this call
#     pdDocClose(doc)
#     return docinfo
# end

# using PDFIO
# getPDFText("electromagnetism_equations.pdf","test.txt")

In [15]:
using Pkg
Pkg.add("SymPy")

  Updating registry at `C:\Users\ses\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %2.4 %.1 %Fetching: [=====>                                   ]  10.4 %14.1 % % [==========>                              ]  24.4 %]  28.6 %Fetching: [==============>                          ]  32.8 %>                        ]  39.7 %.3 %]  45.4 %1 %=======================>                 ]  55.5 %59.3 %Fetching: [===========================>             ]  66.2 %            ]  68.0 %8 %>          ]  74.6 %        ]  79.9 %Fetching: [=================================>       ]  82.1 %7 %9 %]  92.5 %]  95.8 %.2 %>]  99.3 %========================================>]  99.9 % Resolving package versions...
 Installed Adapt ──────── v1.0.1
 Installed RecipesBase ── v0.8.0
 Installed Ratios ─────── v0.4.0
 Installed StatsBase ──── v0.32.1
 Installed CMake ──────── v1.2.0
 Installed FileIO ─────── v

In [16]:
using HTTP, Cascadia, Gumbo, AbstractTrees

url = "https://en.wikipedia.org/wiki/List_of_electromagnetism_equations"
r = HTTP.get(url) # Normally, you'd put a url here, but I couldn't find a way to grab it without having to download it and read it locally
h = parsehtml(String(r)) # Then normally you'd execute this

# # Instead, I'm going to read in the html file as a string and give it to Gumbo
# # h = parsehtml(read("em.html", String))

# # Exploring with the various structure of Gumbo objects:
for elem in PreOrderDFS(h.root) 
    try
        eq = getattr(elem,"alt")
        println(eq)
    catch
    end
end

# println(fieldnames(h.root))
# println(fieldnames(h.root.children))
# println(size(h.root.children))

# aviks code:
# c = matchall(Selector("td:containsOwn(\"CAN:\") + td span"), h.root);
# for x in c
#     println( x.children[1].text )
# end

Solenoid


 q_e = \int \lambda_e \mathrm{d}\ell 
 q_e = \iint \sigma_e \mathrm{d} S 
 q_e = \iiint \rho_e \mathrm{d}V 
C = \mathrm{d}q/\mathrm{d}V\,\!
 I = \mathrm{d}q/\mathrm{d}t \,\!
I = \mathbf{J} \cdot \mathrm{d} \mathbf{S}
 \mathbf{J}_\mathrm{d} = \epsilon_0 \left ( \partial \mathbf{E} / \partial t \right ) = \partial \mathbf{D} / \partial t \,\!
 \mathbf{J}_\mathrm{c} = \rho \mathbf{v} \,\!
\mathbf{E} =\mathbf{F}/q\,\!
\Phi_E = \int_S \mathbf{E} \cdot \mathrm{d} \mathbf{A}\,\!
 \epsilon = \epsilon_r \epsilon_0\,\!
{\displaystyle \mathbf {p} =q\mathbf {a} \,\!}
\mathbf{P} = \mathrm{d} \langle \mathbf{p} \rangle /\mathrm{d} V \,\!
 \mathbf{D} = \epsilon\mathbf{E} = \epsilon_0 \mathbf{E} + \mathbf{P}\,
\Phi_D = \int_S \mathbf{D} \cdot \mathrm{d} \mathbf{A}\,\!
 r_0 \,\!
 r_0 = \infty \,\!
 r_0 = R_\mathrm{earth} \,\!
 V = -\frac{W_{\infty r }}{q} = -\frac{1}{q}\int_\infty^r \mathbf{F} \cdot \mathrm{d} \mathbf{r} = -\int_{r_1}^{r_2} \mathbf{E} \cdot \mathrm{d} \mathbf{r}\,\!
\Delta V

In [1]:
using PyCall

In [2]:
sympy = pyimport("sympy")  #
x = sympy.Symbol("x")      # PyObject x
y = sympy.sin(x)           # PyObject sin(x)
z = y.subs(x, sympy.pi)    # PyObject 0
convert(Float64, z)        # 0.0

0.0

In [14]:
using Conda
Conda.add("process_latex")
# pyimport_conda("process_latex", PKG)
latex = pyimport("process_latex")

┌ Info: Running `conda install -y process_latex` in root environment
└ @ Conda C:\Users\ses\.julia\packages\Conda\CpuvI\src\Conda.jl:112


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - process_latex

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




ErrorException: failed process: Process(setenv(`'C:\Users\ses\.julia\conda\3\Scripts\conda.exe' install -y process_latex`,["PATH=C:\\Users\\ses\\Anaconda3;C:\\Users\\ses\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\ses\\Anaconda3\\Library\\usr\\bin;C:\\Users\\ses\\Anaconda3\\Library\\bin;C:\\Users\\ses\\Anaconda3\\Scripts;C:\\Users\\ses\\Anaconda3\\bin;C:\\Users\\ses\\Anaconda3\\condabin;C:\\Users\\ses\\Anaconda3;C:\\Users\\ses\\Anaconda3\\Library\\mingw-w64\\bin;C:\\Users\\ses\\Anaconda3\\Library\\usr\\bin;C:\\Users\\ses\\Anaconda3\\Library\\bin;C:\\Users\\ses\\Anaconda3\\Scripts;C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\bin;C:\\Program Files\\NVIDIA GPU Computing Toolkit\\CUDA\\v10.1\\libnvvp;C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0;C:\\Program Files (x86)\\NVIDIA Corporation\\PhysX\\Common;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOW